In [1]:
import os
import numpy as np
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
sys.path.insert(0, os.path.abspath('../')) # needed to import src
from src.data_cleaning import WaseemTwitterPreparer, DavidsonTwitterPreparer, FountaTwitterPreparer, \
    GolbeckTwitterPreparer
OUTPUT_DIRECTORY = '../data'

In [2]:
davidson_preparer = DavidsonTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/davidson/davidson_labelled.csv')
davidson_preparer.load_data()
davidson_preparer.prepare_data()

11/21/2020 05:56:28 PM Uncleaned Twitter Davidson Shape: (24783, 6)


In [3]:

waseem_preparer = WaseemTwitterPreparer(verbose=True, path_to_raw=['../data/twitter_datasets/waseem/NAACL_SRW_2016.csv', '../data/twitter_datasets/waseem/NLP+CSS_2016.csv'])
waseem_preparer.load_data()
waseem_preparer.prepare_data()


C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,

In [4]:
golbeck_preparer = GolbeckTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/golbeck/onlineHarassmentDataset.csv')
golbeck_preparer.load_data()
golbeck_preparer.prepare_data()

11/21/2020 05:56:30 PM Uncleaned Twitter Golbeck Shape: (20360, 2)


In [5]:
founta_preparer = FountaTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/founta/hatespeechtwitter.csv')
founta_preparer.load_data()
founta_preparer.prepare_data()

11/21/2020 05:56:30 PM Uncleaned Twitter Founta Shape: (91951, 3)


In [6]:
founta = founta_preparer.get_cleaned_data()
davidson = davidson_preparer.get_cleaned_data()
waseem = waseem_preparer.get_cleaned_data()
golbeck = golbeck_preparer.get_cleaned_data()

In [7]:
founta.loc[:,'original_ds'] = 'founta'
davidson.loc[:,'original_ds'] = 'davidson'
waseem.loc[:,'original_ds'] = 'waseem'
golbeck.loc[:,'original_ds'] = 'golbeck'

In [8]:
print('founta -  shape:', founta.shape, ' - label distribution:', round((founta[founta['is_hate']==1].shape[0] / founta.shape[0]), 3), '- columns:',founta.columns)
print('davidson -  shape:', davidson.shape, ' - label distribution:', round((davidson[davidson['is_hate']==1].shape[0] / davidson.shape[0]), 3), '- columns:', davidson.columns)
print('waseem -  shape:', waseem.shape, ' - label distribution:', round((waseem[waseem['is_hate']==1].shape[0] / waseem.shape[0]), 3), '- columns:', waseem.columns)
print('golbeck -  shape:', golbeck.shape, ' - label distribution:', round((golbeck[golbeck['is_hate']==1].shape[0] / golbeck.shape[0]), 3), '- columns:', golbeck.columns)
# Waiting for Founta to respond to my email as 30k tweets are missing from dataset, many more is_hate

founta -  shape: (91951, 11)  - label distribution: 0.045 - columns: Index(['text', 'maj_label', 'majority_votes', 'is_hate', 'is_abusive',
       'is_harassment', 'cleaned_tweet', 'dialect_prs', 'is_aae_08',
       'is_aae_06', 'original_ds'],
      dtype='object')
davidson -  shape: (24783, 13)  - label distribution: 0.058 - columns: Index(['count', 'hate_speech', 'offensive_language', 'neither', 'class',
       'tweet', 'cleaned_tweet', 'is_hate', 'is_offensive', 'dialect_prs',
       'is_aae_08', 'is_aae_06', 'original_ds'],
      dtype='object')
waseem -  shape: (16631, 12)  - label distribution: 0.204 - columns: Index(['tweet_id', 'label', 'is_racism', 'is_sexism', 'is_hate', 'id', 'text',
       'cleaned_tweet', 'dialect_prs', 'is_aae_08', 'is_aae_06',
       'original_ds'],
      dtype='object')
golbeck -  shape: (19718, 8)  - label distribution: 0.241 - columns: Index(['Code', 'Tweet', 'is_hate', 'cleaned_tweet', 'dialect_prs', 'is_aae_08',
       'is_aae_06', 'original_ds'],


In [9]:
founta['is_harassment'] = np.where(((founta['maj_label']=='abusive') | (founta['maj_label']=='hateful')), 1, 0) #in retrieved tweets, maj label was either abusive or hateful, or spam and normal
davidson['is_harassment'] = np.where(((davidson['is_offensive']==1) | (davidson['is_hate']==1)), 1, 0)
golbeck['is_harassment'] = golbeck['is_hate']
# davidson

In [10]:
# waseem assume that all sexist + racist tweets are harassment, can't assume the inverse (not harassment when not
waseem_harassment = waseem[waseem['is_hate']==1]
waseem_harassment.rename(columns={"is_hate":"is_harassment"}, inplace=True)
waseem_harassment

C:\Users\matan\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,tweet_id,label,is_racism,is_sexism,is_harassment,id,text,cleaned_tweet,dialect_prs,is_aae_08,is_aae_06,original_ds
0,572342978255048705,racism,1,0,1,5.723430e+17,So Drasko just said he was impressed the girls...,So Drasko just said he was impressed the girls...,[0.36626887 0.19337914 0.00452145 0.43583054],0,0,waseem
1,572341498827522049,racism,1,0,1,5.723415e+17,Drasko they didn't cook half a bird you idiot ...,Drasko they didn't cook half a bird you idiot ...,[0.24003915 0.3097326 0.0082669 0.44196135],0,0,waseem
2,572340476503724032,racism,1,0,1,5.723405e+17,Hopefully someone cooks Drasko in the next ep ...,Hopefully someone cooks Drasko in the next ep ...,[0.0824137 0.40979904 0.03430444 0.47348281],0,0,waseem
3,572334712804384768,racism,1,0,1,5.723347e+17,of course you were born in serbia...you're as ...,of course you were born in serbia...you're as ...,[0.11006174 0.21065121 0.06940961 0.60987744],0,0,waseem
4,572332655397629952,racism,1,0,1,5.723327e+17,These girls are the equivalent of the irritati...,These girls are the equivalent of the irritati...,[0.17378829 0.23829187 0.04043375 0.54748609],0,0,waseem
...,...,...,...,...,...,...,...,...,...,...,...,...
16576,603673068691677185,sexism,0,1,1,6.036731e+17,Failure of men to police their own *is* the co...,Failure of men to police their own *is* the co...,[0.11484565 0.13546993 0.12904452 0.6206399 ],0,0,waseem
16602,602697454983749633,both,1,1,1,6.026975e+17,Tokyo Hot n1049 Endless Sex Drive - http://t.c...,Tokyo Hot n1049 Endless Sex Drive - #dailyxLo...,[0.13804527 0.20214447 0.39042414 0.26938612],0,0,waseem
16605,575166549713158144,both,1,1,1,5.751665e+17,baby you can drive my car http://t.co/gpsW3MvA...,baby you can drive my car #asian #asianbabes ...,[0.13968198 0.30230298 0.19563981 0.36237523],0,0,waseem
16611,575447350258216960,sexism,0,1,1,5.754474e+17,http://t.co/Ye7qrymd8Y #GamerGate Kristi Think...,"#GamerGate Kristi Thinks: ""Anita Sarkeesian i...",[0.12811259 0.22216871 0.07926012 0.57045858],0,0,waseem


In [11]:
davidson.rename(columns={"tweet":"text"},inplace=True)
golbeck.rename(columns={"Tweet":"text"},inplace=True)
overlap_cols = list(set(founta.columns).intersection(set(davidson.columns), set(waseem_harassment.columns),set(golbeck.columns)))
overlap_cols

['cleaned_tweet',
 'text',
 'dialect_prs',
 'is_aae_08',
 'is_harassment',
 'original_ds',
 'is_aae_06']

In [12]:
print('founta -  shape:', founta.shape, ' - label distribution:', round((founta[founta['is_harassment']==1].shape[0] / founta.shape[0]), 3),'- n is_aae 0.6 | 0.8:',founta[founta['is_aae_06']==1].shape[0],founta[founta['is_aae_08']==1].shape[0], '- columns:',founta.columns)
print('davidson -  shape:', davidson.shape, ' - label distribution:', round((davidson[davidson['is_harassment']==1].shape[0] / davidson.shape[0]), 3), '- n is_aae 0.6 | 0.8:', davidson[davidson['is_aae_06']==1].shape[0],davidson[davidson['is_aae_08']==1].shape[0],'- columns:', davidson.columns)
print('waseem harassment -  shape:', waseem.shape, ' - label distribution:', round((waseem_harassment[waseem_harassment['is_harassment']==1].shape[0] / waseem_harassment.shape[0]), 3), '- n is_aae 0.6 | 0.8:', waseem_harassment[waseem_harassment['is_aae_06']==1].shape[0],waseem_harassment[waseem_harassment['is_aae_08']==1].shape[0],'- columns:', waseem_harassment.columns)
print('golbeck -  shape:', golbeck.shape, ' - label distribution:', round((golbeck[golbeck['is_harassment']==1].shape[0] / golbeck.shape[0]), 3),'- n is_aae 0.6 | 0.8:',golbeck[golbeck['is_aae_06']==1].shape[0],golbeck[founta['is_aae_08']==1].shape[0], '- columns:', golbeck.columns)

founta -  shape: (91951, 11)  - label distribution: 0.27 - n is_aae 0.6 | 0.8: 1265 85 - columns: Index(['text', 'maj_label', 'majority_votes', 'is_hate', 'is_abusive',
       'is_harassment', 'cleaned_tweet', 'dialect_prs', 'is_aae_08',
       'is_aae_06', 'original_ds'],
      dtype='object')
davidson -  shape: (24783, 14)  - label distribution: 0.832 - n is_aae 0.6 | 0.8: 4878 636 - columns: Index(['count', 'hate_speech', 'offensive_language', 'neither', 'class',
       'text', 'cleaned_tweet', 'is_hate', 'is_offensive', 'dialect_prs',
       'is_aae_08', 'is_aae_06', 'original_ds', 'is_harassment'],
      dtype='object')
waseem harassment -  shape: (16631, 12)  - label distribution: 1.0 - n is_aae 0.6 | 0.8: 12 0 - columns: Index(['tweet_id', 'label', 'is_racism', 'is_sexism', 'is_harassment', 'id',
       'text', 'cleaned_tweet', 'dialect_prs', 'is_aae_08', 'is_aae_06',
       'original_ds'],
      dtype='object')
golbeck -  shape: (19718, 9)  - label distribution: 0.241 - n is_aa

C:\Users\matan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [13]:
merged_df = pd.concat([founta, davidson, waseem_harassment, golbeck])

In [14]:
merged_df = merged_df[overlap_cols]
merged_df.drop_duplicates(inplace=True, subset=['cleaned_tweet']) #7338 duplicate tweets
merged_df.to_csv('../data/twitter_datasets/cleaned/all_harassment_datasets_merged.csv',index=False)

In [15]:
print('merged_df -  shape:', merged_df.shape, ' - label distribution:', round((merged_df[merged_df['is_harassment']==1].shape[0] / merged_df.shape[0]), 3),'- n is_aae 0.6 | 0.8:',merged_df[merged_df['is_aae_06']==1].shape[0],merged_df[merged_df['is_aae_08']==1].shape[0], '- columns:', merged_df.columns)
merged_df

merged_df -  shape: (135023, 7)  - label distribution: 0.391 - n is_aae 0.6 | 0.8: 6205 733 - columns: Index(['cleaned_tweet', 'text', 'dialect_prs', 'is_aae_08', 'is_harassment',
       'original_ds', 'is_aae_06'],
      dtype='object')


,cleaned_tweet,text,dialect_prs,is_aae_08,is_harassment,original_ds,is_aae_06
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,[0.37906185 0.22220451 0.19361946 0.20511418],0,0,founta,0
1,RT : Man it would fucking rule if we had a par...,RT @Papapishu: Man it would fucking rule if we...,[0.2085863 0.21244729 0.09804209 0.48092432],0,1,founta,0
2,It is time to draw close to Him &#128591;&#127...,It is time to draw close to Him &#128591;&#127...,[0.13617963 0.40733362 0.00187648 0.45461027],0,0,founta,0
3,if you notice me start to act different or dis...,if you notice me start to act different or dis...,[0.42431609 0.35141981 0.00202164 0.22224246],0,0,founta,0
4,"Forget unfollowers, I believe in growing. 7 ne...","Forget unfollowers, I believe in growing. 7 ne...",[0.11957159 0.17347329 0.10673621 0.60021892],0,0,founta,0
...,...,...,...,...,...,...,...
19713,Wise words by Ben Shapiro https: / /t.co /ldBj...,Wise words by Ben Shapiro https: / /t.co /ldBj...,[0.14449078 0.16982948 0.24125604 0.4444237 ],0,1,golbeck,0
19714,Women are much more likely to be liberals than...,Women are much more likely to be liberals than...,[0.05411577 0.10954312 0.01288798 0.82345312],0,1,golbeck,0
19715,Would you like me to release to you this King ...,Would you like me to release to you this King ...,[0.17427205 0.34565292 0.09695176 0.38312327],0,1,golbeck,0
19716,You Fucking Nigger You Did It To Yourself You ...,You Fucking Nigger You Did It To Yourself You ...,[0.09649005 0.69237505 0.00770617 0.20342874],0,1,golbeck,0


In [16]:
# 11187/merged_df[merged_df['is_aae']==0].shape[0] # 0.10797428769979152 11187/103608

In [17]:
# 45/merged_df[merged_df['is_aae08']==1].shape[0] #0.06686478454680535 45/673

In [18]:
def _create_test_train_dev_splits(clean_data, tr_size=0.8, dev_test_split=0.5, stratify_label='is_harassment'):
    train, valid_test = train_test_split(clean_data, train_size=tr_size, random_state=42,
                                         stratify=clean_data[[stratify_label]])
    test, valid = train_test_split(valid_test, train_size=dev_test_split, random_state=42,
                                   stratify=valid_test[stratify_label])

    print(f"Train - # of is_harassment: {train[train['is_harassment'] == 1].shape[0]}, total instances: " +
                f"{train.shape[0]}, percent of positive: {train[train['is_harassment'] == 1].shape[0] / train.shape[0]}")
    print(f"Test - # of is_harassment: {test[test['is_harassment'] == 1].shape[0]}, total instances: " +
                f"{test.shape[0]}, percent of positive: {test[test['is_harassment'] == 1].shape[0] / test.shape[0]}")
    print(f"Test - # of is_harassment: {valid[valid['is_harassment'] == 1].shape[0]}, total instances: " +
                    f"{valid.shape[0]}, percent of positive: {valid[valid['is_harassment'] == 1].shape[0] / valid.shape[0]}")

    return train, test, valid

def save_test_train_dev_splits(ds):
    twitter_train, twitter_valid, twitter_test = _create_test_train_dev_splits(ds, tr_size=0.8,
                                                                                    dev_test_split=0.5,
                                                                                    stratify_label='is_harassment')
    # train: 861 7879 0.1093%   test: 214 1962 0.1091    dev: 121 1103 0.1097%

    twitter_train.to_csv('../data/twitter_datasets/combined_harassment/train.csv', index=False)
    twitter_valid.to_csv('../data/twitter_datasets/combined_harassment/dev.csv', index=False)
    twitter_test.to_csv('../data/twitter_datasets/combined_harassment/test.csv', index=False)

In [19]:
save_test_train_dev_splits(merged_df)

C:\Users\matan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train - # of is_harassment: 42273, total instances: 108018, percent of positive: 0.3913514414264289
Test - # of is_harassment: 5284, total instances: 13502, percent of positive: 0.3913494297141164
Test - # of is_harassment: 5284, total instances: 13503, percent of positive: 0.39132044730800564


In [20]:

x = pd.read_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.csv', index_col=0)

FileNotFoundError: [Errno 2] File ../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.csv does not exist: '../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.csv'

In [ ]:
x.to_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv', sep='\t')

In [ ]:
pd.read_json('../data/gab/majority_gab_dataset_25k/test.jsonl', lines=True)

In [ ]:
pd.DataFrame({'a':[1,2],'b':[3,4]}).shape == (2,2)

In [ ]:
import csv
pd.read_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv', delimiter='\t')

f = open('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv')
reader = csv.reader(f, delimiter='\t')
next(reader) # skip header
examples = []
for i, row in enumerate(reader):
    print(row)

In [ ]:
'''
stats
'''

In [ ]:
all_twitter =pd.read_csv('../data/twitter_datasets/cleaned/all_datasets_merged.csv')

In [ ]:
print(all_twitter[all_twitter['is_hate']==1].shape[0])
print(all_twitter[all_twitter['is_hate']==0].shape[0])

In [ ]:
temp = merged_df.drop(columns=['dialect_prs'])
temp[temp.isin([np.nan, np.inf, -np.inf]).any(1)]

